<div style="text-align: justify;">
INTRODUÇÃO: APRESENTAÇÃO DO PROBLEMA E A ESCOLHA DO DATASET PARA O MVP


Hoje vivemos em um cenário, bastante preocupante, onde não e mais possível tolerar ações predatórias contra o meio ambiente, sejam estas os desmatamentos; os incêndios contra vegetações; emissões de poluentes na atmosfera, solos, rios e mares.  Não importa qual seja a ação. Todas geram efeitos adversos e nos cobrarão faturas cada vez mais alta em prazos cada vez mais curtos. As consequências das ações predatórias ambientais não se restringem apenas ao declínio ou mesmo a extinção de espécies da fauna e flora, mas se estende sobre nós, sociedade, expondo-nos a maiores riscos de toda ordem e piorando muito a nossa qualidade de vida. Com esta reflexão em mente, busquei por um conjunto de dados que tratasse dessas mazelas que atingem a natureza e a sociedade simultaneamente.
 

O Instituto Chico Mendes de Conservação da Biodiversidade, ICMBio, disponibiliza no Portal de Dados do Governo Federal, Gov.BR, dados que se alinham ao desejo manifestado acima e, assim, foi selecionado o arquivo intitulado de [Áreas queimadas em unidades de conservação federais](https://dados.gov.br/dados/conjuntos-dados/incendios-em-ucs) para ser utilizado neste MVP.  Outro dataset, localizado no mesmo ambiente, a ser utilizado para dar mais qualidade aos dados é o [Atributos das Unidades de Conservacao Federais]( (https://dados.gov.br/dados/conjuntos-dados/atributos-das-unidades-de-conservacao-federais) .  Mais adiante será aprofundado sobre os conjuntos de dados utilizados aqui na sessão de Metadado e  Catálago, mas deixo já a disposição o [link](https://www.gov.br/icmbio/pt-br/acesso-a-informacao/dados-abertos) que disponibiliza de forma resumida informações sobre as tabelas disponibilizadas.

Após a estruturação dos dados provenientes destes arquivos buscar-se-á as respostas para as seguintes questões: Quais Unidades de Conservação tiveram as maiores áreas queimadas atingidas por incêndios? Entre o intervalo de tempo escolhido, 2018-2023, quais UCs sofreram com as maiores frequências de queimadas? Considerando o Bioma predominante de cada UCs, quais foram as mais afetadas por incêndio? E por fim, tem se a ideia de unir as tabelas de incêndio, que originalmente, estão separadas por ano, para visualizar em gráfico de linha e avaliar como foi a evolução dos incêndios nas Unidades de Conservação ao longo deste período.

Obs1.: os arquivos originais estão formato em excel, sendo necessário a conversão para CSV para o UPLOAD dentro da nuvem do Databricks Community, a versão gratuita para usuários. Para tornar este processo mais ágil, a conversão foi feita no Google Colab onde foi realizada antes uma formatação nos valores dos atributos, transformando letras minúsculas em maiúsculas; retirando os caracteres especiais e acentuações e, no caso, dos títulos das colunas também retirando os espaços em branco, substituindo-os por underline (_). Toda essas ações foram feitas visado evitar problemas na conversão e no carregamento dos arquivos CSV dentro do Databricks. Para verificar como se deu este etapa inicial do trabalho, [ir no Google Colab](https://colab.research.google.com/drive/10mFX7G46NC4IOu58o_YTPW3TM2xAvgrE?authuser=1) .


Obs2:  O código deste notebook desenvolvido também está disponível no [meu GitHub](https://github.com/anapessoapcrj/incedio_uc/blob/main/MVP%20Engenharia%20de%20Dados%20-%20P%C3%B3s-Gradua%C3%A7%C3%A3o%20Ciencia%20de%20Dados%20e%20Analise%20-%20Ana%20Paula%20Pessoa%20da%20Silva%20-%20Jul%202024.py)




1 - UPLOAD dos arquivos CSV para o Unity Catalog do Databricks

<div style="text-align: justify;">
O início da pipeline de dados dentro do Databricks é a partir do UPLOAD dos arquivos csv para este notebook. Como são vários arquivos, foi criado códigos para iterar e agilizar esta etapa. Para isto, foi criado objetos que carregam e salvam dentro de um diretório do Unity Catalog do Databricks. Na mesma linha de código abaixo, foram criadas duas listas, Uma com as urls dos arquivos convertidos em CSV, hospedados no [GitHub](https://raw.githubusercontent.com/anapessoapcrj/incedio_uc) e a outra com os nomes dos arquivos csv. Ambas as listas, além dos referidos objetos, são utilizados na iteração, sendo possível visualizar na execução deste código, onde e como os arquivos CSV foram salvos.

In [0]:


catalog = "incedio"
schema = "dados_uc"
volume = "incedio_historico"
path_volume = f"/Volumes/{catalog}/{schema}/{volume}"

# Lista dos URLs dos arquivos CSV
urls = [
    "https://raw.githubusercontent.com/anapessoapcrj/incedio_uc/main/dadosgerais_uc.csv",
    "https://raw.githubusercontent.com/anapessoapcrj/incedio_uc/main/area_atingida_2023.csv",
    "https://raw.githubusercontent.com/anapessoapcrj/incedio_uc/main/area_atingida_2022.csv",
    "https://raw.githubusercontent.com/anapessoapcrj/incedio_uc/main/area_atingida_2021.csv",
    "https://raw.githubusercontent.com/anapessoapcrj/incedio_uc/main/area_atingida_2020.csv",
    "https://raw.githubusercontent.com/anapessoapcrj/incedio_uc/main/area_atingida_2019.csv",
    "https://raw.githubusercontent.com/anapessoapcrj/incedio_uc/main/area_atingida_2018.csv"
]

# Lista dos nomes dos arquivos CSV
file_names = [
    "dadosgerais_uc.csv",
    "area_atingida_2023.csv",
    "area_atingida_2022.csv",
    "area_atingida_2021.csv",
    "area_atingida_2020.csv",
    "area_atingida_2019.csv",
    "area_atingida_2018.csv"
]

# Loop para baixar cada arquivo CSV e ler como tabela
for url, file_name in zip(urls, file_names):
    # Caminho completo do arquivo no Databricks
    file_path = f"{path_volume}/{file_name}"
    
    # Baixar o arquivo CSV usando dbutils.fs
    dbutils.fs.cp(url, file_path, recurse=False)
    print(f"Arquivo {file_name} baixado para: {file_path}")


# Exemplo de consulta para verificar as tabelas registradas
spark.sql("SHOW TABLES").show()


Arquivo dadosgerais_uc.csv baixado para: /Volumes/incedio/dados_uc/incedio_historico/dadosgerais_uc.csv
Arquivo area_atingida_2023.csv baixado para: /Volumes/incedio/dados_uc/incedio_historico/area_atingida_2023.csv
Arquivo area_atingida_2022.csv baixado para: /Volumes/incedio/dados_uc/incedio_historico/area_atingida_2022.csv
Arquivo area_atingida_2021.csv baixado para: /Volumes/incedio/dados_uc/incedio_historico/area_atingida_2021.csv
Arquivo area_atingida_2020.csv baixado para: /Volumes/incedio/dados_uc/incedio_historico/area_atingida_2020.csv
Arquivo area_atingida_2019.csv baixado para: /Volumes/incedio/dados_uc/incedio_historico/area_atingida_2019.csv
Arquivo area_atingida_2018.csv baixado para: /Volumes/incedio/dados_uc/incedio_historico/area_atingida_2018.csv
+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



<div style="text-align: justify;">
Agora os arquivos serão visualizados como DataFrames atraves do pyspark. Para isto foi criada sessão spark para carregar os csvs em dataframes. Para usar novamente o iterador, desta vez para enxergar como ficará a estrutura de cada arquivo em Dataframe, foi criado objeto que é a lista das tabelas (arquivos_csv) e outros objetos que são opções usadas para o carregamento CSV para DataFrame. A opção FAILFAST, por exemplo, foi utilizada para interromper este processo caso o CSV tenha alguma falha importante que provoque a perda de dados ou deformação na estruturação das tabelas E de fato, o arquivo csv dadosgerais_uc estava com problema, a mensagem de erro indicou onde estava o problema. Feita a correção, no Github, o arquivo foi transferido para Databricks sem problemas. Apos a execução do código abaixo, verificou-se que o tipo de dados de cada coluna se comportou conforme esperado, inclusive as colunas com informações sobre data (foi utilizado encoding UTF-8 para evitar que a data que estava em formato brasileiro se convertesse em string).   

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from delta.tables import DeltaTable

# Cria uma sessão Spark
spark = SparkSession.builder \
    .appName("Carregamento de CSVs em DataFrames") \
    .getOrCreate()

# Lista de caminhos dos arquivos CSV ou nomes de tabelas
lista_tabelas = [
    "/Volumes/incedio/dados_uc/incedio_historico/dadosgerais_uc.csv",
    "/Volumes/incedio/dados_uc/incedio_historico/area_atingida_2023.csv",
    "/Volumes/incedio/dados_uc/incedio_historico/area_atingida_2022.csv",
    "/Volumes/incedio/dados_uc/incedio_historico/area_atingida_2021.csv",
    "/Volumes/incedio/dados_uc/incedio_historico/area_atingida_2020.csv",
    "/Volumes/incedio/dados_uc/incedio_historico/area_atingida_2019.csv",
    "/Volumes/incedio/dados_uc/incedio_historico/area_atingida_2018.csv"
]

# Configurações CSV comuns
file_type = "csv"
infer_schema = "true" #infere a estrutura da tabela 
first_row_is_header = "true"
modo = "FAILFAST" #evita que caso arquivo csv tenha alguma falha ocorra o carregamento dados de forma errônea 
delimiter = ","
encoding = "utf-8"

# Lista para armazenar os DataFrames carregados
dataframes = []

# Iterar sobre cada tabela na lista e imprimir o esquema
for tabela in lista_tabelas:
    # Carregar o arquivo CSV para DataFrame
    df = spark.read.format(file_type) \
      .option("inferSchema", infer_schema) \
      .option("header", first_row_is_header) \
      .option("mode", modo) \
      .option("sep", delimiter) \
      .option("encoding", encoding) \
      .load(tabela)
    
    # Adicionar o DataFrame à lista de DataFrames
    dataframes.append(df)

    # Imprimir o esquema do DataFrame
    print(f"Esquema da tabela {tabela}:")
    df.printSchema()
    print("----------------------------------")

#objetos para cada dataframe
dados_uc = dataframes[0]
incendio_2023_df = dataframes[1]
incendio_2022_df = dataframes[2]
incendio_2021_df = dataframes[3]
incendio_2020_df = dataframes[4]
incendio_2019_df = dataframes[5]
incendio_2018_df = dataframes[6]

# Aqui você pode realizar operações adicionais com cada DataFrame, se necessário


Esquema da tabela /Volumes/incedio/dados_uc/incedio_historico/dadosgerais_uc.csv:
root
 |-- id: integer (nullable = true)
 |-- TIPO_NOME: string (nullable = true)
 |-- TIPO: string (nullable = true)
 |-- Nome_UC: string (nullable = true)
 |-- NGI: string (nullable = true)
 |-- GR: string (nullable = true)
 |-- CNUC: string (nullable = true)
 |-- Atos_legais: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- Bioma_predominante: string (nullable = true)
 |-- Bioma_comp_RL: string (nullable = true)
 |-- Area: double (nullable = true)

----------------------------------
Esquema da tabela /Volumes/incedio/dados_uc/incedio_historico/area_atingida_2023.csv:
root
 |-- ind: integer (nullable = true)
 |-- UC: string (nullable = true)
 |-- NGI: string (nullable = true)
 |-- Incendio: double (nullable = true)
 |-- Queima_prescrita: double (nullable = true)
 |-- Queima_controlada: double (nullable = true)
 |-- Aceiro: double (nullable = true)
 |-- Fogo_natural: double (nullable = true

2 - Transformação dos Dados

<div style="text-align: justify;">
Observando como ficou o nome de algumas colunas, vê se a necessidade de alguma padronização ou correção nos nomes das colunas (ainda que isto já tenha feito no Gooogle Colab), especialmente na tabelas relacionadas a incêndios.   

In [0]:
## Padronização/correção no nome das colunas

incendio_2023_df = incendio_2023_df.withColumnRenamed('Perc_AAF_UC', 'Perc_UC')
incendio_2022_df = incendio_2022_df.withColumnRenamed('Perc_AAF_UC', 'Perc_UC')
incendio_2021_df = incendio_2021_df.withColumnRenamed('`Perc_UC', 'Perc_UC')
incendio_2023_df = incendio_2023_df.withColumnRenamed('Area_total', 'Area_queimada')
incendio_2022_df = incendio_2022_df.withColumnRenamed('Area_total', 'Area_queimada')
incendio_2021_df = incendio_2021_df.withColumnRenamed('_c0', 'ind')


<div style="text-align: justify;">
Area_total foi substituída por Area_queimada nas tabelas incendio_2023_df e incendio_2022_df para ficar congruente com as tabelas dos anos anteriores. Area_total significa as áreas queimadas menos as Áreas de prevenção de incêndios florestais. Também foi substituída Perc_AAF_UC por Perc_UC por entender que o significado é o mesmo, ou seja, percentual de área atingida por incêndio em relação à área da UC

In [0]:
incendio_2023_df.show(n=0)

+---+---+---+--------+----------------+-----------------+------+------------+----------------+---------------+-------------+-------------+-------+-------+
|ind| UC|NGI|Incendio|Queima_prescrita|Queima_controlada|Aceiro|Fogo_natural|Indigena_isolado|Total_prevencao|Total_combate|Area_queimada|Area_UC|Perc_UC|
+---+---+---+--------+----------------+-----------------+------+------------+----------------+---------------+-------------+-------------+-------+-------+
+---+---+---+--------+----------------+-----------------+------+------------+----------------+---------------+-------------+-------------+-------+-------+
only showing top 0 rows



In [0]:
incendio_2022_df.show(n=0)

+---+---+--------+----------------+-----------------+------+------------+---------------+----+-------------+-------+-------+
|ind| UC|Incendio|Queima_prescrita|Queima_controlada|Aceiro|Fogo_natural|Queima_indigena|Raio|Area_queimada|Area_UC|Perc_UC|
+---+---+--------+----------------+-----------------+------+------------+---------------+----+-------------+-------+-------+
+---+---+--------+----------------+-----------------+------+------------+---------------+----+-------------+-------+-------+
only showing top 0 rows



In [0]:
#Verificando o dataframe incendio_2021_df que teve mais colunas renomeadas

incendio_2021_df.show(n=0)

+---+---+-----+------------+-----------------+--------+-----------------+----------------+-----------+------------+-------------+-------+-------+--------------+--------+
|ind| UC|Bioma|Fogo_natural|Queima_indig_isol|Incendio|Queima_controlada|Queima_prescrita|Gestao_raio|Aceiro_negro|Area_queimada|Area_UC|Perc_UC|Data_ult_atual|Satelite|
+---+---+-----+------------+-----------------+--------+-----------------+----------------+-----------+------------+-------------+-------+-------+--------------+--------+
+---+---+-----+------------+-----------------+--------+-----------------+----------------+-----------+------------+-------------+-------+-------+--------------+--------+
only showing top 0 rows



<div style="text-align: justify;">
Também será excluída a coluna id de dados_uc, pois nela já existem três chaves primarias: CNUC, TIPO_NOME e NOME_UC. O primeiro é o Código da Unidade de Conservação, o segundo a Unidade de Conservação com a Categoria abreviada ou com iniciais e a última e Unidade de Conservação escrita por extenso.

In [0]:
dados_uc = dados_uc.drop('id')

In [0]:
dados_uc.show(n=0)

+---------+----+-------+---+---+----+-----------+---+------------------+-------------+----+
|TIPO_NOME|TIPO|Nome_UC|NGI| GR|CNUC|Atos_legais| UF|Bioma_predominante|Bioma_comp_RL|Area|
+---------+----+-------+---+---+----+-----------+---+------------------+-------------+----+
+---------+----+-------+---+---+----+-----------+---+------------------+-------------+----+
only showing top 0 rows



2.1 - Verificação de problemas nas inconsistência dos dados

<div style="text-align: justify;">
Os datasets escolhidos para este MVP, apesar de tabelas, não estão estruturada para ser um banco de dados relacional. Portanto, não podem ser utilizados de imediato para consultas sem antes ter um tratamento corretivo e adaptações.  Este fato é mais perceptível nas tabelas que correspondem às ocorrências de incêndios, onde a escritas dos nomes das Unidades de Conservação ocorre de maneira livre, sem uma preocupação em preservar integridade referencial, sem seguir necessariamente um padrão. Tampouco não têm o CNUC, Código Nacional da Unidade de Conservação, condição ideal para união entre tabelas com colunas com atributos equivalentes.  Posto isto, a seguir teremos consultas para identificar os casos onde não houveram intersecções das Unidades de Conservações registradas nas tabelas de ocorrências de incêndio com a tabela de Atributos das Unidades de Conservação. Ora, as colunas UCs presentes nas tabelas de ocorrências de incêndios deveriam ter 100% de correspondência com a coluna TIPO_NOME da tabela dados_uc, pois este dataset tem o registro atualizado de todas as UCs. No entanto tal condição não se satisfaz, pelo divergência da escrita no nome da Unidades de Conservação, por esta razão é necessário a correção e ajuste. Após a localização dos problemas, os dataframes serão salvos em formato de Tabela Delta onde serão realizados as correções e atualizações.

In [0]:
##Gerando visualizacao temporaria dos dataframes para realizar a consulta pelo spark.sql

dados_uc.createOrReplaceTempView("dados_uc_temp")
incendio_2023_df.createOrReplaceTempView("incendio_2023_temp")
incendio_2022_df.createOrReplaceTempView("incendio_2022_temp")
incendio_2021_df.createOrReplaceTempView("incendio_2021_temp")
incendio_2020_df.createOrReplaceTempView("incendio_2020_temp")
incendio_2019_df.createOrReplaceTempView("incendio_2019_temp")
incendio_2018_df.createOrReplaceTempView("incendio_2018_temp")

In [0]:
# Criando as query para identificar os casos de nao interseccoes inadequadas.
# As consultas onde o problema não existir devera retorna com reposta vazia.  


query1 = """
    SELECT UC
    FROM incendio_2023_temp
    WHERE UC NOT IN
            (SELECT TIPO_NOME 
            FROM dados_uc_temp);
"""

query2 = """
    SELECT UC
    FROM incendio_2022_temp
    WHERE UC NOT IN
            (SELECT TIPO_NOME 
            FROM dados_uc_temp);
"""

query3 = """
    SELECT UC
    FROM incendio_2021_temp
    WHERE UC NOT IN
            (SELECT TIPO_NOME 
            FROM dados_uc_temp);
"""
query4 = """
    SELECT incendio_2020_temp.*
    FROM incendio_2020_temp
    WHERE UC NOT IN
            (SELECT TIPO_NOME 
            FROM dados_uc_temp);
"""
query5 = """
    SELECT UC
    FROM incendio_2019_temp
    WHERE UC NOT IN
            (SELECT TIPO_NOME 
            FROM dados_uc_temp);
"""
query6 = """
    SELECT UC
    FROM incendio_2018_temp
    WHERE UC NOT IN
            (SELECT TIPO_NOME 
            FROM dados_uc_temp);
"""

In [0]:
# Executar a consulta SQL usando spark.sql

resultado_join_INC23_DGER = spark.sql(query1)
resultado_join_INC22_DGER = spark.sql(query2)
resultado_join_INC21_DGER = spark.sql(query3)
resultado_join_INC20_DGER = spark.sql(query4)
resultado_join_INC19_DGER = spark.sql(query5)
resultado_join_INC18_DGER = spark.sql(query6)

Visualizando o resultado para verificar onde serao necessária as correções.

In [0]:
display(resultado_join_INC23_DGER)

UC
APA DA BACIA PARAIBA DO SUL


In [0]:
display(resultado_join_INC22_DGER)

UC
APA BACIA DO PARAIBA DO SUL


In [0]:
display(resultado_join_INC21_DGER)

UC


In [0]:
display(resultado_join_INC20_DGER)

ind,UC,Bioma,Incendio,MIF,Area_queimada,Area_UC,Perc_UC,Data_ult_atual,Satelite
1,APA DA BACIA DO RIO PARNAIBA DO SUL,CERRADO,null,null,187.92,292599.9242,0.064224213,2020-07-20,Landsat / Sentinel


No caso da base de incêndio de 2020 foi necessário verificar todas as colunas, pois não existe APA da Bacia do Rio Parnaíba do Sul. Parece ser um erro de digitação, e ser a APA Bacia Paraíba do Sul. Porém se este for o caso, o Bioma está errado. Não é Cerrado, e sim Mata Atlântica. Será consultada pela palavra PARNAIBA em dados_uc para ver se a área informada sobre APA Bacia do Paraíba do Sul é equivalente ou aproximada. Na tabela de incêndios 2020 o valor informado é 292599.9242 ha. Esta pesquisa será feita mais adiante em conjunto com nomes de UCs não encontrados.

In [0]:
display(resultado_join_INC19_DGER)

UC


In [0]:
display(resultado_join_INC18_DGER)

UC
PARNA DA SERRA DOS ORGAOS
PARNA DAS SEMPRE VIVAS
REBIO DAS NASCENTES DA SERRA DO CACHIMBO


<div style="text-align: justify;">
É possível verificar que a APA DA BACIA PARAIBA DO SUL, PARNA DA SERRA DOS ORGAOS, PARNA DAS SEMPRE VIVAS E REBIO DAS NASCENTES DA SERRA DO CACHIMBO não foram encontrado na tabela Dados Gerais UC. Vamos pesquisar agora em Dados_Gerais_UC como está escrito estas UCs para substituir a digitação de maneira igual a Dados_Gerais_UC para que seja possível a perfeita junção entre as tabelas de incêndio com a de atributos.

In [0]:
query7 = """
    SELECT TIPO_NOME, NOME_UC, Area
    FROM dados_uc_temp
    WHERE NOME_UC LIKE '%CACHIMBO%' OR NOME_UC LIKE '%SEMPRE%' OR NOME_UC LIKE '%ORGAOS%' OR NOME_UC LIKE '%ORGAOS%' OR NOME_UC LIKE '%PARAIBA%' OR NOME_UC LIKE '%PARNAIBA%'
"""

In [0]:
pesquisa_nomes_uc = spark.sql(query7)

<div style="text-align: justify;">
A suspeita de erro na digitação em APA da Bacia do Rio Parnaíba do Sul ao inves de APA Bacia da Paraíba do Sul foi confirmada, pois o área da UC informada na tabela de incêndio "bate" com a informada na tabela, 292599.9242 ha, com Atributos das Unidades de Conservação, conforme é possível ver em tabela abaixo. 

In [0]:
display(pesquisa_nomes_uc)

TIPO_NOME,NOME_UC,Area
APA BACIA PARAIBA DO SUL,AREA DE PROTECAO AMBIENTAL BACIA DO PARAIBA DO SUL,292599.924223634
APA DELTA DO PARNAIBA,AREA DE PROTECAO AMBIENTAL DELTA DO PARNAIBA,309593.770698495
PARNA DA SERRA DOS ORGAOS,PARQUE NACIONAL DA SERRA DOS ORGAOS,19850.3065728119
PARNA DAS NASCENTES DO RIO PARNAIBA,PARQUE NACIONAL DAS NASCENTES DO RIO PARNAIBA,749774.175082093
PARNA DAS SEMPRE-VIVAS,PARQUE NACIONAL DAS SEMPRE-VIVAS,124155.899321356
REBIO NASCENTES DA SERRA DO CACHIMBO,RESERVA BIOLOGICA NASCENTES DA SERRA DO CACHIMBO,342195.772904465
RESEX MARINHA DO DELTA DO PARNAIBA,RESERVA EXTRATIVISTA MARINHA DO DELTA DO PARNAIBA,27022.0734188426
RESEX VERDE PARA SEMPRE,RESERVA EXTRATIVISTA VERDE PARA SEMPRE,1289379.74234782


<div style="text-align: justify;">
Agora que foi finalizado a verificação dos problemas, os dataframes serão salvos para formato de tabela delta.  As correções e ajustes necessários serão feito na sequência. A forma de salvar deste arquivos foram definidos com o modo de sobreescrever tabelas e seus esquemas, assim é possivel reexecutar as linhas de código sem problema interrupção.

In [0]:
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

In [0]:
dados_uc.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("/FileStore/tables/uc_atualizado_jun_2024")

In [0]:
incendio_2023_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("/FileStore/tables/uc_2023_incendio")

In [0]:
incendio_2022_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("/FileStore/tables/uc_2022_incendio")

In [0]:
incendio_2021_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("/FileStore/tables/uc_2021_incendio")

In [0]:
incendio_2020_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("/FileStore/tables/uc_2020_incendio")

In [0]:
incendio_2019_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("/FileStore/tables/uc_2019_incendio")

In [0]:
incendio_2018_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("/FileStore/tables/uc_2018_incendio")

Correção da digitação da Unidades de Conservação e outros.
  
Será criado uma lista com o caminho onde estão as tabelas Delta e depois em cima desta lista iterar as correções.

In [0]:
# Lista de caminhos das tabelas Delta Lake
caminhos_tabelas_delta = ["/FileStore/tables/uc_2023_incendio", "/FileStore/tables/uc_2022_incendio", "/FileStore/tables/uc_2021_incendio", "/FileStore/tables/uc_2020_incendio", "/FileStore/tables/uc_2019_incendio","/FileStore/tables/uc_2018_incendio",]

# Carregar todas as tabelas Delta Lake
delta_tables = [DeltaTable.forPath(spark, caminho) for caminho in caminhos_tabelas_delta]

In [0]:
# Iterar sobre todas as tabelas Delta Lake e executar a atualização
for delta_tab in delta_tables:
    delta_tab.update(
        condition="UC = 'APA BACIA DO PARAIBA DO SUL'",
        set={"UC": "'APA BACIA PARAIBA DO SUL'"}
    )

In [0]:
for delta_tab in delta_tables:
    delta_tab.update(
        condition="UC = 'APA DA BACIA PARAIBA DO SUL'",
        set={"UC": "'APA BACIA PARAIBA DO SUL'"}
    )

In [0]:
for delta_tab in delta_tables:
    delta_tab.update(
        condition="UC = 'APA DA BACIA DO RIO PARNAIBA DO SUL'",
        set={"UC": "'APA BACIA PARAIBA DO SUL'"}
    )

<div style="text-align: justify;">
Nem todas as tabelas com registros de incêndio têm a coluna Bioma, não sendo possível iterar a sua correção. Neste caso, esta ação sera feita pontualmente na tabela de incêndios em 2020 para corrigir o Bioma da APA Bacia Paraíba do Sul. O ajuste será substituir CERRADO por MATA ATLANTICA.

In [0]:
caminhos_tabelas_delta2 = "/FileStore/tables/uc_2020_incendio"

delta_table2 = DeltaTable.forPath(spark, caminhos_tabelas_delta2)

delta_table2.update(
        condition="UC == 'APA BACIA PARAIBA DO SUL'",
        set={"Bioma": "'MATA ATLANTICA'"}
)

In [0]:
for delta_tab in delta_tables:
    delta_tab.update(
        condition="UC = 'REBIO DAS NASCENTES DA SERRA DO CACHIMBO'",
        set={"UC": "'REBIO NASCENTES DA SERRA DO CACHIMBO'"}
    )

In [0]:
for delta_tab in delta_tables:
    delta_tab.update(
        condition="UC = 'PARNA DAS SEMPRE VIVAS'",
        set={"UC": "'PARNA DAS SEMPRE-VIVAS'"}
    )

In [0]:
for delta_tab in delta_tables:
    delta_tab.update(
        condition="UC = 'PARNA DA SERRA DOS ORGAOS '",
        set={"UC": "'PARNA DA SERRA DOS ORGAOS'"}
    )

2.2 - Terminado a correção dos atributos das tabelas, daremos inicio a processo de construção de Metadados. Etapa fundamental para a organização e governança dos dados.  

<div style="text-align: justify;">
Para realização desta etapa foi necessário registrar as tabelas deltas no catálogo Spark. Ao fazer isto, é criado um catálogo onde possível inserir comentários que descrevem as características de cada coluna. Esta ação fez-se pelo uso combinado dos comandos ALTER TABLE e CHANGE COLUMN pelo pyspark, além do comando CREATE TABLE e USING delta LOCATION para apontar a tabela delta de referência para criação do catálogo. Outro acréscimo foi sobre informações gerais de cada tabela: como a origem dos dados; quem é o controlador e quem o mantenedor; o período em que este dado é atualizado e quando foi a última atualização; quem modificou o dado e em que período ocorreu. Para esta operação, foi usado a linguagem SQL aplicando em conjunto os comandos ALTER TABLE DELTA e SET TBLPROPERTIES. Estas informações ficam em Table Properties do catálogo da tabela delta disponível na última linha do catálogo.

No percurso deste trabalho foi percebido a necessidade de criar uma coluna ano para cada tabela relacionada a incêndio. Na etapa final deste trabalho ficará bem evidente que a criação do campo ano é fundamental para verificar a evolução dos incêndio nas UCs entre os anos de 2018 à 2023.  Optou-se em adicionar estas colunas já dentro do formato delta através da linguagem SQL e os comentários relativos ao ano foram adicionados nos catálogos das tabelas. Os comandos utilizados foram ALTER TABLE; ADD COLUMN para criar a coluna com o tipo de dado Inteiro e, para preencher com os valores predefinido - os anos dos incêndios, UPDATE; SET. A sintaxe para as tabelas ficaram desta forma na primeira linha de código: ALTER TABLE delta./FileStore/tables/uc_2023_incendio ADD COLUMN ano INT;. Na segunda linha: UPDATE delta./FileStore/tables/uc_2023_incendio SET ano = 2023;. Este padrão se repetiu para as tabelas incêndios de outros anos.

In [0]:
caminho_tabela_delta = "/FileStore/tables/uc_atualizado_jun_2024"

# Registrar a tabela Delta no catálogo do Spark
spark.sql(f"CREATE TABLE info_dados_uc1 USING delta LOCATION '{caminho_tabela_delta}'")

Out[37]: DataFrame[]

In [0]:
spark.sql(f"ALTER TABLE info_dados_uc1 CHANGE COLUMN TIPO_NOME TIPO_NOME STRING COMMENT 'Unidade de Conservacao com a Categoria abreviada ou com iniciais'")
spark.sql(f"ALTER TABLE info_dados_uc1 CHANGE COLUMN TIPO TIPO STRING COMMENT 'Categorias de Unidade de Conservacao abreviado ou com iniciais. Ver mais em Lei 9985/2000-SNUC'")
spark.sql(f"ALTER TABLE info_dados_uc1 CHANGE COLUMN Nome_UC Nome_UC STRING COMMENT 'Unidade de Conservacao com descricao completa'")
spark.sql(f"ALTER TABLE info_dados_uc1 CHANGE COLUMN NGI NGI STRING COMMENT 'Nucleo de Gestao Integrada'")
spark.sql(f"ALTER TABLE info_dados_uc1 CHANGE COLUMN GR GR STRING COMMENT 'Gerencia Regional'")
spark.sql(f"ALTER TABLE info_dados_uc1 CHANGE COLUMN CNUC CNUC STRING COMMENT 'Codigo CNUC (MMA)'")
spark.sql(f"ALTER TABLE info_dados_uc1 CHANGE COLUMN Atos_legais Atos_legais STRING COMMENT 'Atos legais (Criacao ou redefinicao)'")
spark.sql(f"ALTER TABLE info_dados_uc1 CHANGE COLUMN UF UF STRING COMMENT 'UF de Abrangencia'")
spark.sql(f"ALTER TABLE info_dados_uc1 CHANGE COLUMN Bioma_predominante Bioma_predominante STRING COMMENT 'Bioma (IBGE 1:250mil). Para as UCs que ocorrem em mais de um bioma considera-se a que abrange 50% ou mais de seu territorio'")
spark.sql(f"ALTER TABLE info_dados_uc1 CHANGE COLUMN Bioma_comp_RL Bioma_comp_RL STRING COMMENT 'Bioma para fins de Compensacao de Reserva Legal'")
spark.sql(f"ALTER TABLE info_dados_uc1 CHANGE COLUMN Area Area DOUBLE COMMENT 'Area em hectares'")

Out[38]: DataFrame[]

In [0]:
spark.sql(f"""
    ALTER TABLE delta.`{caminho_tabela_delta}`
    SET TBLPROPERTIES (
        ' Info' = 'Tabela modificada cujo dados sao provenientes do dataset Atributos das Unidades de Conservação Federais disponivel em https://dados.gov.br/dados/conjuntos-dados/atributos-das-unidades-de-conservacao-federais .  Controlador: ICMBio. Mantenedor: Coordenação-Geral de Consolidação Territorial - CGTER. Frequencia de atualizacao: Quadrimestral. Ultima atualizacao: Junho/2024. Modificado por Ana Paula Pessoa da Silva para entrega de trabalho de Eng. De Dados da Pos-Graduacao em Ciencia de Dados e Analiticas do PUC-Rio de Janeiro entre periodo de junho a julho deste ano (2024)  '
    )
""")


Out[96]: DataFrame[]

Visualizando o Catálago pertinentes aos Atributos de Unidades de Conservação Federais

In [0]:
%sql
DESCRIBE EXTENDED delta.`/FileStore/tables/uc_atualizado_jun_2024`;

col_name,data_type,comment
TIPO_NOME,string,Unidade de Conservacao com a Categoria abreviada ou com iniciais
TIPO,string,Categorias de Unidade de Conservacao abreviado ou com iniciais. Ver mais em Lei 9985/2000-SNUC
Nome_UC,string,Unidade de Conservacao com descricao completa
NGI,string,Nucleo de Gestao Integrada
GR,string,Gerencia Regional
CNUC,string,Codigo CNUC (MMA)
Atos_legais,string,Atos legais (Criacao ou redefinicao)
UF,string,UF de Abrangencia
Bioma_predominante,string,Bioma (IBGE 1:250mil). Para as UCs que ocorrem em mais de um bioma considera-se a que abrange 50% ou mais de seu territorio
Bioma_comp_RL,string,Bioma para fins de Compensacao de Reserva Legal


In [0]:
%sql

ALTER TABLE delta.`/FileStore/tables/uc_2023_incendio` ADD COLUMN ano INT;

In [0]:
%sql

UPDATE delta.`/FileStore/tables/uc_2023_incendio` SET ano = 2023;

num_affected_rows
132


In [0]:
caminho_ano_2023 = "/FileStore/tables/uc_2023_incendio"

# Registrar a tabela Delta no catálogo do Spark
spark.sql(f"CREATE TABLE info_incendio_2023 USING delta LOCATION '{caminho_ano_2023}'")

Out[42]: DataFrame[]

In [0]:
spark.sql(f"ALTER TABLE info_incendio_2023 CHANGE COLUMN ind ind INT COMMENT 'index gerado'")
spark.sql(f"ALTER TABLE info_incendio_2023 CHANGE COLUMN UC UC STRING COMMENT 'Unidade de Conservacao com a Categoria abreviada ou com iniciais'")
spark.sql(f"ALTER TABLE info_incendio_2023 CHANGE COLUMN NGI NGI STRING COMMENT 'Nucleo de Gestao Integrada'")
spark.sql(f"ALTER TABLE info_incendio_2023 CHANGE COLUMN Incendio Incendio DOUBLE COMMENT 'Area em hectares da UC atingida por incendio'")
spark.sql(f"ALTER TABLE info_incendio_2023 CHANGE COLUMN Queima_prescrita Queima_prescrita DOUBLE COMMENT 'Queimada prescrita em hectares'")
spark.sql(f"ALTER TABLE info_incendio_2023 CHANGE COLUMN Queima_controlada Queima_controlada DOUBLE COMMENT 'Queimada controlada em hectares'")
spark.sql(f"ALTER TABLE info_incendio_2023 CHANGE COLUMN Aceiro Aceiro DOUBLE COMMENT 'Area queimada em hectares por fogo controlado para servir como barreira natural contra incendios florestais'")
spark.sql(f"ALTER TABLE info_incendio_2023 CHANGE COLUMN Fogo_natural Fogo_natural DOUBLE COMMENT 'Area queimada em hectares cuja as causas do fogo foram naturais'")
spark.sql(f"ALTER TABLE info_incendio_2023 CHANGE COLUMN Indigena_isolado Indigena_isolado DOUBLE COMMENT 'Area queimada em hectares por grupos indigenas isolados'")
spark.sql(f"ALTER TABLE info_incendio_2023 CHANGE COLUMN Total_prevencao Total_prevencao DOUBLE COMMENT 'Area em hectares com prevencao de incendios'")
spark.sql(f"ALTER TABLE info_incendio_2023 CHANGE COLUMN Total_combate Total_combate DOUBLE COMMENT 'Area em hectares de combate a incendios'")
spark.sql(f"ALTER TABLE info_incendio_2023 CHANGE COLUMN Area_queimada Area_queimada DOUBLE COMMENT 'Area Queimada total em hectares. Nao e incluido o total de area com prevencao de incendios florestais'")
spark.sql(f"ALTER TABLE info_incendio_2023 CHANGE COLUMN Area_UC Area_UC DOUBLE COMMENT 'Area em hectares da UC'")
spark.sql(f"ALTER TABLE info_incendio_2023 CHANGE COLUMN Perc_UC Perc_UC DOUBLE COMMENT 'Percentual de area afetada pelo incendio em relacao a area da UC'")
spark.sql(f"ALTER TABLE info_incendio_2023 CHANGE COLUMN ano ano INT COMMENT 'Ano da ocorrencia do incendio'")

Out[43]: DataFrame[]

In [0]:
spark.sql(f"""
    ALTER TABLE delta.`{caminho_ano_2023}`
    SET TBLPROPERTIES (
        ' Info' = 'Tabela modificada cujo dados sao provenientes do dataset Áreas Queimadas em Unidades de Conservação Federais disponivel em https://dados.gov.br/dados/conjuntos-dados/incendios-em-ucs .  Controlador ICMBio. Mantenedor: Divisão de Informações Geoespaciais e Monitoramento - DGEO. Frequencia de atualizacao: Anual. Ultima atualizacao: Outubro/2023. Modificado por Ana Paula Pessoa da Silva para entrega de trabalho de Eng. De Dados da Pos-Graduacao em Ciencia de Dados e Analiticas do PUC-Rio de Janeiro entre periodo de junho a julho deste ano (2024)  '
    )
""")


Out[44]: DataFrame[]

Será acrescentado a coluna ano para as tabelas que registram ocorrência de incêndio para o próxima etapa consulta e análise da evolução temporal dos incêndios nas UC.  

Visualizando o catálago pertinentes aos Dados de Registro de Incendios nas UCs de 2023  

In [0]:
%sql
DESCRIBE EXTENDED delta.`/FileStore/tables/uc_2023_incendio`;


col_name,data_type,comment
ind,int,index gerado
UC,string,Unidade de Conservacao com a Categoria abreviada ou com iniciais
NGI,string,Nucleo de Gestao Integrada
Incendio,double,Area em hectares da UC atingida por incendio
Queima_prescrita,double,Queimada prescrita em hectares
Queima_controlada,double,Queimada controlada em hectares
Aceiro,double,Area queimada em hectares por fogo controlado para servir como barreira natural contra incendios florestais
Fogo_natural,double,Area queimada em hectares cuja as causas do fogo foram naturais
Indigena_isolado,double,Area queimada em hectares por grupos indigenas isolados
Total_prevencao,double,Area em hectares com prevencao de incendios


In [0]:
%sql
ALTER TABLE delta.`/FileStore/tables/uc_2022_incendio`
ADD COLUMN ano INT;

In [0]:
%sql
UPDATE delta.`/FileStore/tables/uc_2022_incendio`
SET ano = 2022;

num_affected_rows
97


In [0]:
caminho_ano_2022 = "/FileStore/tables/uc_2022_incendio"

# Registrar a tabela Delta no catálogo do Spark
spark.sql(f"CREATE TABLE info_incendio_2022 USING delta LOCATION '{caminho_ano_2022}'")

Out[48]: DataFrame[]

In [0]:
spark.sql(f"ALTER TABLE info_incendio_2022 CHANGE COLUMN ind ind INT COMMENT 'index gerado'")
spark.sql(f"ALTER TABLE info_incendio_2022 CHANGE COLUMN UC UC STRING COMMENT 'Unidade de Conservacao com a Categoria abreviada ou com iniciais'")
spark.sql(f"ALTER TABLE info_incendio_2022 CHANGE COLUMN Incendio Incendio DOUBLE COMMENT 'Area em hectares da UC atingida por incendio'")
spark.sql(f"ALTER TABLE info_incendio_2022 CHANGE COLUMN Queima_prescrita Queima_prescrita DOUBLE COMMENT 'Queimada prescrita em hectares'")
spark.sql(f"ALTER TABLE info_incendio_2022 CHANGE COLUMN Queima_controlada Queima_controlada DOUBLE COMMENT 'Queimada controlada em hectares'")
spark.sql(f"ALTER TABLE info_incendio_2022 CHANGE COLUMN Aceiro Aceiro DOUBLE COMMENT 'Area queimada em hectares por fogo controlado para servir como barreira natural contra incendios florestais'")
spark.sql(f"ALTER TABLE info_incendio_2022 CHANGE COLUMN Fogo_natural Fogo_natural DOUBLE COMMENT 'Area queimada em hectares cuja as causas do fogo foram naturais'")
spark.sql(f"ALTER TABLE info_incendio_2022 CHANGE COLUMN Queima_indigena Queima_indigena DOUBLE COMMENT 'Area queimada em hectares por grupos indigenas'")
spark.sql(f"ALTER TABLE info_incendio_2022 CHANGE COLUMN Raio Raio DOUBLE COMMENT 'Area queimada em hectares causado por raio'")
spark.sql(f"ALTER TABLE info_incendio_2022 CHANGE COLUMN Area_queimada Area_queimada DOUBLE COMMENT 'Area Queimada total em hectares. Nao e incluido o total de area com prevencao de incendios florestais'")
spark.sql(f"ALTER TABLE info_incendio_2022 CHANGE COLUMN Area_UC Area_UC DOUBLE COMMENT 'Area em hectares da UC'")
spark.sql(f"ALTER TABLE info_incendio_2022 CHANGE COLUMN Perc_UC Perc_UC DOUBLE COMMENT 'Percentual de area afetada pelo incendio em relacao a area da UC'")
spark.sql(f"ALTER TABLE info_incendio_2022 CHANGE COLUMN ano ano INT COMMENT 'Ano da ocorrencia do incendio'")

Out[50]: DataFrame[]

In [0]:
spark.sql(f"""
    ALTER TABLE delta.`{caminho_ano_2022}`
    SET TBLPROPERTIES (
        ' Info' = 'Tabela modificada cujo dados sao provenientes do dataset Áreas Queimadas em Unidades de Conservação Federais disponivel em https://dados.gov.br/dados/conjuntos-dados/incendios-em-ucs .  Controlador ICMBio. Mantenedor: Divisão de Informações Geoespaciais e Monitoramento - DGEO. Frequencia de atualizacao: Anual. Ultima atualizacao: Outubro/2023. Modificado por Ana Paula Pessoa da Silva para entrega de trabalho de Eng. De Dados da Pos-Graduacao em Ciencia de Dados e Analiticas do PUC-Rio de Janeiro entre periodo de junho a julho deste ano (2024)  '
    )
""")

Out[51]: DataFrame[]

Visualizando o catálago pertinentes aos Dados de Registro de Incendios nas UCs de 2022  

In [0]:
%sql
DESCRIBE EXTENDED delta.`/FileStore/tables/uc_2022_incendio`;


col_name,data_type,comment
ind,int,index gerado
UC,string,Unidade de Conservacao com a Categoria abreviada ou com iniciais
Incendio,double,Area em hectares da UC atingida por incendio
Queima_prescrita,double,Queimada prescrita em hectares
Queima_controlada,double,Queimada controlada em hectares
Aceiro,double,Area queimada em hectares por fogo controlado para servir como barreira natural contra incendios florestais
Fogo_natural,double,Area queimada em hectares cuja as causas do fogo foram naturais
Queima_indigena,double,Area queimada em hectares por grupos indigenas
Raio,double,Area queimada em hectares causado por raio
Area_queimada,double,Area Queimada total em hectares. Nao e incluido o total de area com prevencao de incendios florestais


In [0]:
%sql
ALTER TABLE delta.`/FileStore/tables/uc_2021_incendio` ADD COLUMN ano INT;

In [0]:
%sql
UPDATE delta.`/FileStore/tables/uc_2021_incendio`
SET ano = 2021;

num_affected_rows
44


In [0]:
caminho_ano_2021 = "/FileStore/tables/uc_2021_incendio"

# Registrar a tabela Delta no catálogo do Spark
spark.sql(f"CREATE TABLE info_incendio_2021 USING delta LOCATION '{caminho_ano_2021}'")

Out[55]: DataFrame[]

In [0]:
spark.sql(f"ALTER TABLE info_incendio_2021 CHANGE COLUMN ind ind INT COMMENT 'index gerado'")
spark.sql(f"ALTER TABLE info_incendio_2021 CHANGE COLUMN UC UC STRING COMMENT 'Unidade de Conservacao com a Categoria abreviada ou com iniciais'")
spark.sql(f"ALTER TABLE info_incendio_2021 CHANGE COLUMN Bioma Bioma STRING COMMENT 'Conjunto de vida vegetal e animal, constituído pelo agrupamento de tipos de vegetação que são próximos e que podem ser identificados em nível regional, com condições de geologia e clima semelhantes e que, historicamente, sofreram os mesmos processos de formação da paisagem - https://educa.ibge.gov.br/criancas/brasil/nosso-territorio/19635-ecossistemas.html'")
spark.sql(f"ALTER TABLE info_incendio_2021 CHANGE COLUMN Fogo_natural Fogo_natural DOUBLE COMMENT 'Area queimada em hectares cuja as causas do fogo foram naturais'")
spark.sql(f"ALTER TABLE info_incendio_2021 CHANGE COLUMN Queima_indig_isol Queima_indig_isol DOUBLE COMMENT 'Area queimada em hectares por grupos indigenas isolados'")
spark.sql(f"ALTER TABLE info_incendio_2021 CHANGE COLUMN Incendio Incendio DOUBLE COMMENT 'Area em hectares da UC atingida por incendio'")
spark.sql(f"ALTER TABLE info_incendio_2021 CHANGE COLUMN Queima_controlada Queima_controlada DOUBLE COMMENT 'Queimada controlada em hectares'")
spark.sql(f"ALTER TABLE info_incendio_2021 CHANGE COLUMN Queima_prescrita Queima_prescrita DOUBLE COMMENT 'Queimada prescrita em hectares'")
spark.sql(f"ALTER TABLE info_incendio_2021 CHANGE COLUMN Gestao_raio Gestao_raio DOUBLE COMMENT 'Area queimada em hectares causado por raio'")
spark.sql(f"ALTER TABLE info_incendio_2021 CHANGE COLUMN Aceiro_negro Aceiro_negro DOUBLE COMMENT 'Area queimada em hectares por fogo controlado para servir como barreira natural contra incendios florestais'")
spark.sql(f"ALTER TABLE info_incendio_2021 CHANGE COLUMN Area_queimada Area_queimada DOUBLE COMMENT 'Area Queimada em hectares'")
spark.sql(f"ALTER TABLE info_incendio_2021 CHANGE COLUMN Area_UC Area_UC DOUBLE COMMENT 'Area em hectares da UC'")
spark.sql(f"ALTER TABLE info_incendio_2021 CHANGE COLUMN Perc_UC Perc_UC DOUBLE COMMENT 'Percentual de area afetada pelo incendio em relacao a area da UC'")
spark.sql(f"ALTER TABLE info_incendio_2021 CHANGE COLUMN Data_ult_atual Data_ult_atual DATE COMMENT 'Data da ultima atualizacao do conjunto de dados original'")
spark.sql(f"ALTER TABLE info_incendio_2021 CHANGE COLUMN Satelite Satelite STRING COMMENT 'Imagem de satelite utilizada para identificar focos de incendios florestais'")
spark.sql(f"ALTER TABLE info_incendio_2021 CHANGE COLUMN ano ano INT COMMENT 'Ano da ocorrencia do incendio'")

Out[56]: DataFrame[]

In [0]:
spark.sql(f"""
    ALTER TABLE delta.`{caminho_ano_2021}`
    SET TBLPROPERTIES (
        ' Info' = 'Tabela modificada cujo dados sao provenientes do dataset Áreas Queimadas em Unidades de Conservação Federais disponivel em https://dados.gov.br/dados/conjuntos-dados/incendios-em-ucs .  Controlador ICMBio. Mantenedor: Divisão de Informações Geoespaciais e Monitoramento - DGEO. Frequencia de atualizacao: Anual. Ultima atualizacao: Outubro/2023. Modificado por Ana Paula Pessoa da Silva para entrega de trabalho de Eng. De Dados da Pos-Graduacao em Ciencia de Dados e Analiticas do PUC-Rio de Janeiro entre periodo de junho a julho deste ano (2024)  '
    )
""")

Out[57]: DataFrame[]

Visualizando o catálago pertinentes aos Dados de Registro de Incendios nas UCs de 2021

In [0]:
%sql
DESCRIBE EXTENDED delta.`/FileStore/tables/uc_2021_incendio`;

col_name,data_type,comment
ind,int,index gerado
UC,string,Unidade de Conservacao com a Categoria abreviada ou com iniciais
Bioma,string,"Conjunto de vida vegetal e animal, constituído pelo agrupamento de tipos de vegetação que são próximos e que podem ser identificados em nível regional, com condições de geologia e clima semelhantes e que, historicamente, sofreram os mesmos processos de formação da paisagem - https://educa.ibge.gov.br/criancas/brasil/nosso-territorio/19635-ecossistemas.html"
Fogo_natural,double,Area queimada em hectares cuja as causas do fogo foram naturais
Queima_indig_isol,double,Area queimada em hectares por grupos indigenas isolados
Incendio,double,Area em hectares da UC atingida por incendio
Queima_controlada,double,Queimada controlada em hectares
Queima_prescrita,double,Queimada prescrita em hectares
Gestao_raio,double,Area queimada em hectares causado por raio
Aceiro_negro,double,Area queimada em hectares por fogo controlado para servir como barreira natural contra incendios florestais


In [0]:
%sql
ALTER TABLE delta.`/FileStore/tables/uc_2020_incendio` ADD COLUMN ano INT;

In [0]:
%sql
UPDATE delta.`/FileStore/tables/uc_2020_incendio`
SET ano = 2020;

num_affected_rows
47


In [0]:
caminho_ano_2020 = "/FileStore/tables/uc_2020_incendio"

# Registrar a tabela Delta no catálogo do Spark
spark.sql(f"CREATE TABLE info_incendio_2020 USING delta LOCATION '{caminho_ano_2020}'")

Out[61]: DataFrame[]

In [0]:
spark.sql(f"ALTER TABLE info_incendio_2020 CHANGE COLUMN ind ind INT COMMENT 'index gerado'")
spark.sql(f"ALTER TABLE info_incendio_2020 CHANGE COLUMN UC UC STRING COMMENT 'Unidade de Conservacao com a Categoria abreviada ou com iniciais'")
spark.sql(f"ALTER TABLE info_incendio_2020 CHANGE COLUMN Bioma Bioma STRING COMMENT 'Conjunto de vida vegetal e animal, constituído pelo agrupamento de tipos de vegetação que são próximos e que podem ser identificados em nível regional, com condições de geologia e clima semelhantes e que, historicamente, sofreram os mesmos processos de formação da paisagem - https://educa.ibge.gov.br/criancas/brasil/nosso-territorio/19635-ecossistemas.html'")
spark.sql(f"ALTER TABLE info_incendio_2020 CHANGE COLUMN Incendio Incendio DOUBLE COMMENT 'Area em hectares da UC atingida por incendio'")
spark.sql(f"ALTER TABLE info_incendio_2020 CHANGE COLUMN MIF MIF DOUBLE COMMENT 'Area em hectares do Manejo Imtegrado do Fogo'")
spark.sql(f"ALTER TABLE info_incendio_2020 CHANGE COLUMN Area_queimada Area_queimada DOUBLE COMMENT 'Area Queimada em hectares'")
spark.sql(f"ALTER TABLE info_incendio_2020 CHANGE COLUMN Area_UC Area_UC DOUBLE COMMENT 'Area em hectares da UC'")
spark.sql(f"ALTER TABLE info_incendio_2020 CHANGE COLUMN Perc_UC Perc_UC DOUBLE COMMENT 'Percentual de area afetada pelo incendio em relacao a area da UC'")
spark.sql(f"ALTER TABLE info_incendio_2020 CHANGE COLUMN Data_ult_atual Data_ult_atual DATE COMMENT 'Data da ultima atualizacao do conjunto de dados original'")
spark.sql(f"ALTER TABLE info_incendio_2020 CHANGE COLUMN Satelite Satelite STRING COMMENT 'Imagem de satelite utilizada para identificar focos de incendios florestais'")
spark.sql(f"ALTER TABLE info_incendio_2020 CHANGE COLUMN ano ano INT COMMENT 'Ano da ocorrencia do incendio'")

Out[62]: DataFrame[]

In [0]:
spark.sql(f"""
    ALTER TABLE delta.`{caminho_ano_2020}`
    SET TBLPROPERTIES (
        ' Info' = 'Tabela modificada cujo dados sao provenientes do dataset Áreas Queimadas em Unidades de Conservação Federais disponivel em https://dados.gov.br/dados/conjuntos-dados/incendios-em-ucs .  Controlador ICMBio. Mantenedor: Divisão de Informações Geoespaciais e Monitoramento - DGEO. Frequencia de atualizacao: Anual. Ultima atualizacao: Outubro/2023. Modificado por Ana Paula Pessoa da Silva para entrega de trabalho de Eng. De Dados da Pos-Graduacao em Ciencia de Dados e Analiticas do PUC-Rio de Janeiro entre periodo de junho a julho deste ano (2024)  '
    )
""")

Out[63]: DataFrame[]

Visualizando o catálago pertinentes aos Dados de Registro de Incendios nas UCs de 2020

In [0]:
%sql
DESCRIBE EXTENDED delta.`/FileStore/tables/uc_2020_incendio`;

col_name,data_type,comment
ind,int,index gerado
UC,string,Unidade de Conservacao com a Categoria abreviada ou com iniciais
Bioma,string,"Conjunto de vida vegetal e animal, constituído pelo agrupamento de tipos de vegetação que são próximos e que podem ser identificados em nível regional, com condições de geologia e clima semelhantes e que, historicamente, sofreram os mesmos processos de formação da paisagem - https://educa.ibge.gov.br/criancas/brasil/nosso-territorio/19635-ecossistemas.html"
Incendio,double,Area em hectares da UC atingida por incendio
MIF,double,Area em hectares do Manejo Imtegrado do Fogo
Area_queimada,double,Area Queimada em hectares
Area_UC,double,Area em hectares da UC
Perc_UC,double,Percentual de area afetada pelo incendio em relacao a area da UC
Data_ult_atual,date,Data da ultima atualizacao do conjunto de dados original
Satelite,string,Imagem de satelite utilizada para identificar focos de incendios florestais


In [0]:
%sql
ALTER TABLE delta.`/FileStore/tables/uc_2019_incendio` ADD COLUMN ano INT;

In [0]:
%sql
UPDATE delta.`/FileStore/tables/uc_2019_incendio`
SET ano = 2019;

num_affected_rows
61


In [0]:
caminho_ano_2019 = "/FileStore/tables/uc_2019_incendio"

# Registrar a tabela Delta no catálogo do Spark
spark.sql(f"CREATE TABLE info_incendio_2019 USING delta LOCATION '{caminho_ano_2019}'")

Out[67]: DataFrame[]

In [0]:
spark.sql(f"ALTER TABLE info_incendio_2019 CHANGE COLUMN ind ind INT COMMENT 'index gerado'")
spark.sql(f"ALTER TABLE info_incendio_2019 CHANGE COLUMN UC UC STRING COMMENT 'Unidade de Conservacao com a Categoria abreviada ou com iniciais'")
spark.sql(f"ALTER TABLE info_incendio_2019 CHANGE COLUMN Bioma Bioma STRING COMMENT 'Conjunto de vida vegetal e animal, constituído pelo agrupamento de tipos de vegetação que são próximos e que podem ser identificados em nível regional, com condições de geologia e clima semelhantes e que, historicamente, sofreram os mesmos processos de formação da paisagem - https://educa.ibge.gov.br/criancas/brasil/nosso-territorio/19635-ecossistemas.html'")
spark.sql(f"ALTER TABLE info_incendio_2019 CHANGE COLUMN Area_queimada Area_queimada DOUBLE COMMENT 'Area Queimada em hectares'")
spark.sql(f"ALTER TABLE info_incendio_2019 CHANGE COLUMN Area_UC Area_UC DOUBLE COMMENT 'Area em hectares da UC'")
spark.sql(f"ALTER TABLE info_incendio_2019 CHANGE COLUMN Perc_UC Perc_UC DOUBLE COMMENT 'Percentual de area afetada pelo incendio em relacao a area da UC'")
spark.sql(f"ALTER TABLE info_incendio_2019 CHANGE COLUMN Data_ult_atual Data_ult_atual DATE COMMENT 'Data da ultima atualizacao do conjunto de dados original'")
spark.sql(f"ALTER TABLE info_incendio_2019 CHANGE COLUMN Satelite Satelite STRING COMMENT 'Imagem de satelite utilizada para identificar focos de incendios florestais'")
spark.sql(f"ALTER TABLE info_incendio_2019 CHANGE COLUMN ano ano INT COMMENT 'Ano da ocorrencia do incendio'")

Out[68]: DataFrame[]

In [0]:
spark.sql(f"""
    ALTER TABLE delta.`{caminho_ano_2019}`
    SET TBLPROPERTIES (
        ' Info' = 'Tabela modificada cujo dados sao provenientes do dataset Áreas Queimadas em Unidades de Conservação Federais disponivel em https://dados.gov.br/dados/conjuntos-dados/incendios-em-ucs .  Controlador ICMBio. Mantenedor: Divisão de Informações Geoespaciais e Monitoramento - DGEO. Frequencia de atualizacao: Anual. Ultima atualizacao: Outubro/2023. Modificado por Ana Paula Pessoa da Silva para entrega de trabalho de Eng. De Dados da Pos-Graduacao em Ciencia de Dados e Analiticas do PUC-Rio de Janeiro entre periodo de junho a julho deste ano (2024)  '
    )
""")

Out[69]: DataFrame[]

Visualizando o catálago pertinentes aos Dados de Registro de Incendios nas UCs de 2019

In [0]:
%sql
DESCRIBE EXTENDED delta.`/FileStore/tables/uc_2019_incendio`;

col_name,data_type,comment
ind,int,index gerado
UC,string,Unidade de Conservacao com a Categoria abreviada ou com iniciais
Bioma,string,"Conjunto de vida vegetal e animal, constituído pelo agrupamento de tipos de vegetação que são próximos e que podem ser identificados em nível regional, com condições de geologia e clima semelhantes e que, historicamente, sofreram os mesmos processos de formação da paisagem - https://educa.ibge.gov.br/criancas/brasil/nosso-territorio/19635-ecossistemas.html"
Area_queimada,double,Area Queimada em hectares
Area_UC,double,Area em hectares da UC
Perc_UC,double,Percentual de area afetada pelo incendio em relacao a area da UC
Data_ult_atual,date,Data da ultima atualizacao do conjunto de dados original
Satelite,string,Imagem de satelite utilizada para identificar focos de incendios florestais
ano,int,Ano da ocorrencia do incendio
,,


In [0]:
%sql
ALTER TABLE delta.`/FileStore/tables/uc_2018_incendio` ADD COLUMN ano INT;

In [0]:
%sql
UPDATE delta.`/FileStore/tables/uc_2018_incendio`
SET ano = 2018;

num_affected_rows
43


In [0]:
caminho_ano_2018 = "/FileStore/tables/uc_2018_incendio"

# Registrar a tabela Delta no catálogo do Spark
spark.sql(f"CREATE TABLE info_incendio_2018 USING delta LOCATION '{caminho_ano_2018}'")

Out[73]: DataFrame[]

In [0]:
spark.sql(f"ALTER TABLE info_incendio_2018 CHANGE COLUMN ind ind INT COMMENT 'index gerado'")
spark.sql(f"ALTER TABLE info_incendio_2018 CHANGE COLUMN UC UC STRING COMMENT 'Unidade de Conservacao com a Categoria abreviada ou com iniciais'")
spark.sql(f"ALTER TABLE info_incendio_2018 CHANGE COLUMN Bioma Bioma STRING COMMENT 'Conjunto de vida vegetal e animal, constituído pelo agrupamento de tipos de vegetação que são próximos e que podem ser identificados em nível regional, com condições de geologia e clima semelhantes e que, historicamente, sofreram os mesmos processos de formação da paisagem - https://educa.ibge.gov.br/criancas/brasil/nosso-territorio/19635-ecossistemas.html'")
spark.sql(f"ALTER TABLE info_incendio_2018 CHANGE COLUMN Area_queimada Area_queimada DOUBLE COMMENT 'Area Queimada em hectares'")
spark.sql(f"ALTER TABLE info_incendio_2018 CHANGE COLUMN Area_UC Area_UC DOUBLE COMMENT 'Area em hectares da UC'")
spark.sql(f"ALTER TABLE info_incendio_2018 CHANGE COLUMN Perc_UC Perc_UC DOUBLE COMMENT 'Percentual de area afetada pelo incendio em relacao a area da UC'")
spark.sql(f"ALTER TABLE info_incendio_2018 CHANGE COLUMN Data_ult_atual Data_ult_atual DATE COMMENT 'Data da ultima atualizacao'")
spark.sql(f"ALTER TABLE info_incendio_2018 CHANGE COLUMN Satelite Satelite STRING COMMENT 'Imagem de satelite utilizada para identificar focos de incendios florestais'")
spark.sql(f"ALTER TABLE info_incendio_2018 CHANGE COLUMN ano ano INT COMMENT 'Ano da ocorrencia do incendio'")

Out[74]: DataFrame[]

In [0]:
spark.sql(f"""
    ALTER TABLE delta.`{caminho_ano_2018}`
    SET TBLPROPERTIES (
        ' Info' = 'Tabela modificada cujo dados sao provenientes do dataset Áreas Queimadas em Unidades de Conservação Federais disponivel em https://dados.gov.br/dados/conjuntos-dados/incendios-em-ucs .  Controlador ICMBio. Mantenedor: Divisão de Informações Geoespaciais e Monitoramento - DGEO. Frequencia de atualizacao: Anual. Ultima atualizacao: Outubro/2023. Modificado por Ana Paula Pessoa da Silva para entrega de trabalho de Eng. De Dados da Pos-Graduacao em Ciencia de Dados e Analiticas do PUC-Rio de Janeiro entre periodo de junho a julho deste ano (2024)  '
    )
""")

Out[75]: DataFrame[]

Visualizando o catálago pertinentes aos Dados de Registro de Incendios nas UCs de 2018

In [0]:
%sql
DESCRIBE EXTENDED delta.`/FileStore/tables/uc_2018_incendio`;

col_name,data_type,comment
ind,int,index gerado
UC,string,Unidade de Conservacao com a Categoria abreviada ou com iniciais
Bioma,string,"Conjunto de vida vegetal e animal, constituído pelo agrupamento de tipos de vegetação que são próximos e que podem ser identificados em nível regional, com condições de geologia e clima semelhantes e que, historicamente, sofreram os mesmos processos de formação da paisagem - https://educa.ibge.gov.br/criancas/brasil/nosso-territorio/19635-ecossistemas.html"
Area_queimada,double,Area Queimada em hectares
Area_UC,double,Area em hectares da UC
Perc_UC,double,Percentual de area afetada pelo incendio em relacao a area da UC
Data_ult_atual,date,Data da ultima atualizacao
Satelite,string,Imagem de satelite utilizada para identificar focos de incendios florestais
ano,int,Ano da ocorrencia do incendio
,,


3 - Extração de Dados - Consultas

<div style="text-align: justify;">
Agora é o momento de realizar as consultas para extrair as informações a respeito das ocorrências de incêndios. Para isto, as tabelas deltas serão lidas como janelas temporárias.  Além disso, serão criadas tabelas temporárias de ocorrencia de incêndio com mesmo esquema para viabilizar a reunião destas em uma, só através do comando UNION ALL do SQL      

In [0]:

spark.read.format("delta").load("dbfs:/FileStore/tables/uc_2018_incendio").createOrReplaceTempView("incendio_2018")
spark.read.format("delta").load("dbfs:/FileStore/tables/uc_2019_incendio").createOrReplaceTempView("incendio_2019")
spark.read.format("delta").load("dbfs:/FileStore/tables/uc_2020_incendio").createOrReplaceTempView("incendio_2020")
spark.read.format("delta").load("dbfs:/FileStore/tables/uc_2021_incendio").createOrReplaceTempView("incendio_2021")
spark.read.format("delta").load("dbfs:/FileStore/tables/uc_2022_incendio").createOrReplaceTempView("incendio_2022")
spark.read.format("delta").load("dbfs:/FileStore/tables/uc_2023_incendio").createOrReplaceTempView("incendio_2023")
spark.read.format("delta").load("dbfs:/FileStore/tables/uc_atualizado_jun_2024").createOrReplaceTempView("dados_uc")

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW DADOS_UC AS
SELECT *
FROM delta.`dbfs:/FileStore/tables/uc_atualizado_jun_2024`;

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW INCEND_2023 AS
SELECT UC, Area_queimada, Perc_UC, ano
FROM delta.`dbfs:/FileStore/tables/uc_2023_incendio`;

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW INCEND_2022 AS
SELECT UC, Area_queimada, Perc_UC, ano
FROM delta.`dbfs:/FileStore/tables/uc_2022_incendio`;

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW INCEND_2021 AS
SELECT UC, Area_queimada, Perc_UC, ano
FROM delta.`dbfs:/FileStore/tables/uc_2021_incendio`;

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW INCEND_2020 AS
SELECT UC, Area_queimada, Perc_UC, ano
FROM delta.`dbfs:/FileStore/tables/uc_2020_incendio`;

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW INCEND_2019 AS
SELECT UC, Area_queimada, Perc_UC, ano
FROM delta.`dbfs:/FileStore/tables/uc_2019_incendio`;

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW INCEND_2018 AS
SELECT UC, Area_queimada, Perc_UC, ano
FROM delta.`dbfs:/FileStore/tables/uc_2018_incendio`;

In [0]:
%sql

CREATE OR REPLACE TEMPORARY VIEW HISTORICO AS
SELECT *
FROM INCEND_2023
UNION ALL
SELECT *
FROM INCEND_2022
UNION ALL
SELECT *
FROM INCEND_2021
UNION ALL
SELECT *
FROM INCEND_2020
UNION ALL
SELECT *
FROM INCEND_2019
UNION ALL
SELECT *
FROM INCEND_2018


Aproveitando que foi criado uma tabela temporaria única para as ocorrências de incêndio será feito uma outra acrescentando os atributos da Tabela com Informações Gerais das Unidades de Conservação.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW HISTORICO_DADOS_UC AS
SELECT CNUC, TIPO_NOME, NOME_UC, TIPO, Bioma_predominante, UF, Area_queimada, Perc_UC, ano
FROM DADOS_UC
INNER JOIN HISTORICO
ON TIPO_NOME = UC


In [0]:
%sql
--este esquema seria o ideal para a tabela de incêndio.  Uma tabela única com uma coluna ano que identifique a ocorrência do incêndio e com id das UC, a CNUC
SELECT *
FROM HISTORICO_DADOS_UC

CNUC,TIPO_NOME,NOME_UC,TIPO,Bioma_predominante,UF,Area_queimada,Perc_UC,ano
0000.00.0020,APA CARSTE DE LAGOA SANTA,AREA DE PROTECAO AMBIENTAL CARSTE DE LAGOA SANTA,APA,CERRADO,MG,162.8936726,0.43,2023
0000.00.0002,APA CAVERNAS DO PERUACU,AREA DE PROTECAO AMBIENTAL CAVERNAS DO PERUACU,APA,CERRADO,MG,388.4169684,0.34,2023
0000.00.0003,APA DA BACIA DO RIO DESCOBERTO,AREA DE PROTECAO AMBIENTAL DA BACIA DO RIO DESCOBERTO,APA,CERRADO,GO;DF,713.0488549,1.97,2023
0000.00.1521,APA BACIA PARAIBA DO SUL,AREA DE PROTECAO AMBIENTAL BACIA DO PARAIBA DO SUL,APA,MATA ATLANTICA,MG;RJ;SP,545.3385567,0.19,2023
0000.00.0004,APA DA BACIA DO RIO SAO BARTOLOMEU,AREA DE PROTECAO AMBIENTAL DA BACIA DO RIO SAO BARTOLOMEU,APA,CERRADO,DF,184.5129023,0.22,2023
0000.00.0005,APA DA BACIA DO RIO SAO JOAO/MICO-LEAO-DOURADO,AREA DE PROTECAO AMBIENTAL DA BACIA DO RIO SAO JOAO/MICO-LEAO-DOURADO,APA,MATA ATLANTICA,RJ,530.6036077,0.36,2023
0000.00.0006,APA DA BALEIA FRANCA,AREA DE PROTECAO AMBIENTAL DA BALEIA FRANCA,APA,MATA ATLANTICA,SC,106.6427139,0.07,2023
0000.00.0008,APA DA CHAPADA DO ARARIPE,AREA DE PROTECAO AMBIENTAL DA CHAPADA DO ARARIPE,APA,CAATINGA,PI;CE;PE,172.7711811,0.02,2023
0000.00.0010,APA DA REGIAO SERRANA DE PETROPOLIS,AREA DE PROTECAO AMBIENTAL DA REGIAO SERRANA DE PETROPOLIS,APA,MATA ATLANTICA,RJ,165.4501,0.34,2023
0000.00.0011,APA DA SERRA DA MANTIQUEIRA,AREA DE PROTECAO AMBIENTAL DA SERRA DA MANTIQUEIRA,APA,MATA ATLANTICA,MG;RJ;SP,247.6535041,0.06,2023



Vamos iniciar as consultas.  Primeira: pergunta quais foram as cinco UCs mais afetadas por incêndios no ano de 2018? Apresentar a resposta com o bioma predominante.

In [0]:
%sql
-- Somar a área queimada e ter uma média do percentual da area UC atingida
SELECT NOME_UC, Bioma_predominante, SUM(Area_queimada) AS Maiores_Areas_Queimadas_2018, AVG(PERC_UC) AS Media_Percentual_AreaUC_Atingida_2018
FROM HISTORICO_DADOS_UC
WHERE ano = 2018
GROUP BY NOME_UC, Bioma_predominante
ORDER BY Maiores_Areas_Queimadas_2018 DESC
LIMIT 5;

NOME_UC,Bioma_predominante,Maiores_Areas_Queimadas_2018,Media_Percentual_AreaUC_Atingida_2018
ESTACAO ECOLOGICA SERRA GERAL DO TOCANTINS,CERRADO,141042.1699,19.94691726963209
PARQUE NACIONAL DO ARAGUAIA,CERRADO,138454.9113,24.92361372505252
PARQUE NACIONAL DAS NASCENTES DO RIO PARNAIBA,CERRADO,84336.75101,11.643426412972797
ESTACAO ECOLOGICA NIQUIA,AMAZONIA,71161.66,24.987360889905716
PARQUE NACIONAL DA SERRA DA CANASTRA,CERRADO,44544.02514,22.51870498056206


E quais foram as cinco UCs mais afetadas por incêndios no ano de 2019? Apresentar a resposta com o bioma predominante.

In [0]:
%sql
-- Somar a área queimada e ter uma média do percentual da area UC atingida
SELECT NOME_UC, Bioma_predominante, SUM(Area_queimada) AS Maiores_Areas_Queimadas_2019, AVG(PERC_UC) AS Media_Percentual_AreaUC_Atingida_2019
FROM HISTORICO_DADOS_UC
WHERE ano = 2019
GROUP BY NOME_UC, Bioma_predominante
ORDER BY Maiores_Areas_Queimadas_2019 DESC
LIMIT 5;

NOME_UC,Bioma_predominante,Maiores_Areas_Queimadas_2019,Media_Percentual_AreaUC_Atingida_2019
PARQUE NACIONAL DO ARAGUAIA,CERRADO,329410.2527,59.29796076735725
PARQUE NACIONAL DAS NASCENTES DO RIO PARNAIBA,CERRADO,196611.26411856242,27.143905336548155
ESTACAO ECOLOGICA SERRA GERAL DO TOCANTINS,CERRADO,172347.31867264572,24.37424792036406
FLORESTA NACIONAL DE RORAIMA,AMAZONIA,54500.40523772123,32.129235615823525
FLORESTA NACIONAL DO JAMANXIM,AMAZONIA,50227.77364861864,3.858636511549678


E quais foram as cinco UCs mais afetadas por incêndios no ano de 2020? Apresentar a resposta com o bioma predominante.

In [0]:
%sql
-- Somar a área queimada e ter uma média do percentual da area UC atingida
SELECT NOME_UC, Bioma_predominante, SUM(Area_queimada) AS Maiores_Areas_Queimadas_2020, AVG(PERC_UC) AS Media_Percentual_AreaUC_Atingida_2020
FROM HISTORICO_DADOS_UC
WHERE ano = 2020
GROUP BY NOME_UC, Bioma_predominante
ORDER BY Maiores_Areas_Queimadas_2020 DESC
LIMIT 5;

NOME_UC,Bioma_predominante,Maiores_Areas_Queimadas_2020,Media_Percentual_AreaUC_Atingida_2020
ESTACAO ECOLOGICA SERRA GERAL DO TOCANTINS,CERRADO,261143.2174,36.932222524681485
PARQUE NACIONAL DO ARAGUAIA,CERRADO,254503.3305,45.81316553212965
PARQUE NACIONAL DAS NASCENTES DO RIO PARNAIBA,CERRADO,205335.9804,27.38637675361551
PARQUE NACIONAL DO PANTANAL MATO-GROSSENSE,PANTANAL,132679.51403299998,97.61398688681554
PARQUE NACIONAL GRANDE SERTAO VEREDAS,CERRADO,94140.23766,40.77874492042363


E quais foram as cinco UCs mais afetadas por incêndios no ano de 2021? Apresentar a resposta com o bioma predominante.

In [0]:
%sql
-- Somar a área queimada e ter uma média do percentual da area UC atingida
SELECT NOME_UC, Bioma_predominante, SUM(Area_queimada) AS Maiores_Areas_Queimadas_2021, AVG(PERC_UC) AS Media_Percentual_AreaUC_Atingida_2021
FROM HISTORICO_DADOS_UC
WHERE ano = 2021
GROUP BY NOME_UC, Bioma_predominante
ORDER BY Maiores_Areas_Queimadas_2021 DESC
LIMIT 5;

NOME_UC,Bioma_predominante,Maiores_Areas_Queimadas_2021,Media_Percentual_AreaUC_Atingida_2021
PARQUE NACIONAL DO ARAGUAIA,CERRADO,243856.304718,43.896593535835905
PARQUE NACIONAL DAS NASCENTES DO RIO PARNAIBA,CERRADO,159477.244095,21.27003694944607
ESTACAO ECOLOGICA SERRA GERAL DO TOCANTINS,CERRADO,136794.97771,19.346252246008504
REFUGIO DE VIDA SILVESTRE DAS VEREDAS DO OESTE BAIANO,CERRADO,52223.35,40.78338510689775
FLORESTA NACIONAL DO JAMANXIM,AMAZONIA,44461.549547,3.415659225557721


E quais foram as cinco UCs mais afetadas por incêndios no ano de 2022? Apresentar a resposta com o bioma predominante.

In [0]:
%sql
-- Somar a área queimada e ter uma média do percentual da area UC atingida
SELECT NOME_UC, Bioma_predominante, SUM(Area_queimada) AS Maiores_Areas_Queimadas_2022, AVG(PERC_UC) AS Media_Percentual_AreaUC_Atingida_2022
FROM HISTORICO_DADOS_UC
WHERE ano = 2022
GROUP BY NOME_UC, Bioma_predominante
ORDER BY Maiores_Areas_Queimadas_2022 DESC
LIMIT 5;

NOME_UC,Bioma_predominante,Maiores_Areas_Queimadas_2022,Media_Percentual_AreaUC_Atingida_2022
ESTACAO ECOLOGICA SERRA GERAL DO TOCANTINS,CERRADO,297816.7325890216,42.12
PARQUE NACIONAL DO ARAGUAIA,CERRADO,203292.5169332762,36.59
PARQUE NACIONAL DAS NASCENTES DO RIO PARNAIBA,CERRADO,165651.8899353112,22.09
PARQUE NACIONAL DA CHAPADA DOS VEADEIROS,CERRADO,64619.17947,26.86
PARQUE NACIONAL DA CHAPADA DAS MESAS,CERRADO,63967.66737698494,39.99


E quais foram as cinco UCs mais afetadas por incêndios no ano de 2023? Apresentar a resposta com o bioma predominante.

In [0]:
%sql

SELECT NOME_UC, Bioma_predominante, SUM(Area_queimada) AS Maiores_Areas_Queimadas_2023, AVG(PERC_UC) AS Media_Percentual_AreaUC_Atingida_2023
FROM HISTORICO_DADOS_UC
WHERE ano = 2023
GROUP BY NOME_UC, Bioma_predominante
ORDER BY Maiores_Areas_Queimadas_2023 DESC
LIMIT 5;

NOME_UC,Bioma_predominante,Maiores_Areas_Queimadas_2023,Media_Percentual_AreaUC_Atingida_2023
ESTACAO ECOLOGICA SERRA GERAL DO TOCANTINS,CERRADO,172718.7743,24.43
PARQUE NACIONAL DO ARAGUAIA,CERRADO,155650.9251,28.02
AREA DE PROTECAO AMBIENTAL DOS MEANDROS DO RIO ARAGUAIA,CERRADO,59709.49648,16.62
PARQUE NACIONAL DO PANTANAL MATO-GROSSENSE,PANTANAL,53291.3074,39.21
PARQUE NACIONAL DA SERRA DA CANASTRA,CERRADO,46780.91214,23.63


<div style="text-align: justify;">
Pelas respostas é possível afirmar que o Bioma da UCs mais atingido por incêndios é o Cerrado. O Pantanal chama atenção pelo elevado percentual de área atingida em relação à área total da UC, o Parque Nacional do Pantanal Mato-Grossense. Em 2020 foi próximo de 100%, algo que impressiona. Em 2023, o percentual foi menor, próximo de 40%, mas ainda assim, signifativo. Com relação a persistência do problema entre o período de 2018-2023, a Estação Ecológica Serra Geral do Tocantins se manteve entre na liderança juntamente com O Parque Nacional do Araguaia.  São duas UCs que exigem maior atenção para a questão analisada aqui.  

<div style="text-align: justify;">
São muita perguntas que podem ser feitas, mas vamos encerrar duas perguntas finais. A primeira é para obter uma de quais as Unidades Conservação entre o período de 2018-2023 não registraram ocorrências de incêndios.  É importante destacar que por não existir uma coluna específica para data de criação legal das Unidades de Conservação, o resultado deve passar por um crivo depois para identificar quais são Unidades de Conservação mais recentes. Isto porque obviamente as UCs mais recentes não terão registros de incêndios antes de sua criação, o que não significa que o lugar não tenha sofrido deste problema.  Apesar desta ressalva, é melhor este tipo consulta do que verificar qual foi a menor área atingida por queimada em Unidade de Conservação. O motivo disto é muito simples: uma Unidade de Conservação server para proteção ambiental integral ou para manejo sustentável, ou seja, este lugares não é para sofrer com incêndios florestais, não importa a dimensão da área.

A outra consulta é soma total das áreas queimadas por ano para visualização de como evolui as ocorrências de incêndios ao longo do referido período.  

In [0]:
%sql
--UCs não atingidas por incêndios. Observação: verificar a data de criação da Unidade de Conservação para validação da resposta
SELECT TIPO_NOME, Atos_legais
FROM DADOS_UC
WHERE TIPO_NOME NOT IN 
                (SELECT TIPO_NOME
                FROM HISTORICO_DADOS_UC)

TIPO_NOME,Atos_legais
APA ANHATOMIRIM,DEC 528 de 20/05/1992
APA COSTA DAS ALGAS,DEC S/N de 17/06/2010
APA DA ARARINHA AZUL,DEC 9.402 de 05/06/2018
APA DA BARRA DO RIO MAMANGUAPE,DEC 924 de 10/09/1993; DEC S;N 7/04/1998
APA DA COSTA DOS CORAIS,DEC S/N de 23/10/1997
APA DE FERNANDO DE NORONHA - ROCAS - SAO PEDRO E SAO PAULO,DEC 92.755 de 05/06/1986
APA DE GUARAQUECABA,DEC 90.883 de 31/01/1985
APA DE PIACABUCU,DEC 88.421 de 21/06/1983
APA DO ARQUIPELAGO DE SAO PEDRO E SAO PAULO,DEC 9.313 de 19/03/2018
APA DO ARQUIPELAGO DE TRINDADE E MARTIM VAZ,DEC 9.312 de 19/03/2018


In [0]:
%sql
--Gráfico que registra a evolução das áreas Queimadas entre o período de 2018 à 2023
SELECT SUM(Area_queimada) AS AREA_QUEIM_ANO, ano
FROM HISTORICO_DADOS_UC
GROUP BY ano

AREA_QUEIM_ANO,ano
915223.419561141,2023
1236528.0334871493,2022
1018696.7040600003,2021
1491426.500577,2020
1206317.7612448896,2019
650356.2418569999,2018


Databricks visualization. Run in Databricks to view.

Pelos gráfico acima é possível perceber que entre 2018 à 2020, houve uma expansão significativa da área atingidas por queimmadas nas UCs. A partir de 2020 à 2023, a área atingida diminui e fica oscilando entre 2021 à 2023.  Infelizmente a queda da área atingida não foi o suficiente para voltar ao número de área afetada em 2018. E possívelmente quando tivermos os dados consolidados das áreas atingidas deste ano, teremos um aumento significativo, vide as notícias a respeito do incêndio do Pantanal em 2024. 

<div style="text-align: justify;">
4 - Encerramento do Trabalho e Conclusão 

O trabalho se iniciou pelo tratamento dos arquivos em formato xlsx no Colab do Google Driver. Após isto, os arquivos foram convertidos em CSV, armazenados no GitHub e foram carregados para dentro da Plataforma Databricks. Após o upload, foram verificado quais ajustes e correções necessárias que foram realizadas já com as tabelas em formato delta. O ajuste final foi acréscimo da coluna "ano" para cada tabela de ocorrências, o que foi fundamental na etapa final de união de várias tabelas em uma apenas.  Durante a consulta constatou-se que o bioma mais afetado nas UCs é o Cerrado.  O Pantanal não esteve presente em todos os anos do  (2018 à 2023), mas quando aparece, especialmente em 2020, surge com percentual alarmante de área afetada em relação a área de abragência da UC. No gráfico de evolução, é possível perceber que 2020 foi o pior ano com as maiores áreas atingidas pelo fogo.  

Foi interessante desenvolver este projeto e trabalhar com dados que me são muito caros. Acredito que os objetivos do trabalho foram alcançados e que foi cumprido todo o ciclo de vida de produção do dado, apesar do curtíssimo espaço de tempo para aprender a utilizar o Databricks, uma plataforma que até então desconhecia. Mas reconheço que oferece bastantes recursos mesmo na versão gratuita, dois que cito é a possibilidade de usar diferentes linguagens em um só notebook e a outra é a facilidade de gerar a visualização dos dados pelos gráficos. Pontos que entendo que o banco de dados que construi precisa de melhorias são: criação de uma coluna com a data da criação da UC e criação de nova tabela onde tenhas as UCs e as UFs, estas como valores únicos. Oportunamente serão adotadas estas melhorias, pois a ideia é continuar alimentado este banco de dados relativos aos incêndios a medida que a ICMBio for atualizando os dados no site GOV.BR.  

Encerro saudando o trabalho dos servidores do Ministério do Meio Ambiente e fico na torcida para que os políticos se sensibilizem para a questão ambiental, que é urgente, e parem de cortar orçamento para a Pasta Ambiental. O futuro do nosso país agradece. 